In [1]:
# 比较单目标算法
import numpy as np
np.random.seed(987)
import geatpy as ea
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib
import time
"""============================目标函数============================"""    
def aim(x):                         # 传入种群染色体矩阵解码后的基因表现型矩阵
    global best_acc
    fitness = np.zeros(NIND)        # 记录适应度矩阵    
    for i in range(NIND):
        select = []                 # 获取选择到特征的对应下标
        target_stock = pd.read_csv("./dataset/n_"+data+".csv")
        y_train= target_stock[1:length1]["trend"]
        y_test = target_stock[length1:length2-1]["trend"]
        y_valid = target_stock[length2:-1]["trend"]
        for j in range(Lind):
            if x[i][j]==1:
               select.append(j)
        target_stock = target_stock.iloc[:,select]
        x_train = target_stock[0:length1-1]
        x_test = target_stock[length1-1:length2-2]
        x_valid = target_stock[length2-1:-2]
        # 归一化
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.fit_transform(x_test)
        x_valid = scaler.fit_transform(x_valid)
        # svc_clf = RandomForestClassifier()
        svc_clf = XGBClassifier()
        svc_clf.fit(x_train,y_train)
        y_pre=svc_clf.predict(x_test)
        y_pre_v = svc_clf.predict(x_valid)
        
        print("TEST_ACC: %.3f"%(accuracy_score(y_test,y_pre)*100))
        print("VALID_ACC: %.3f"%(accuracy_score(y_valid,y_pre_v)*100))
        print("*"*30)
        if(accuracy_score(y_valid,y_pre_v)>best_acc):
            best_acc = accuracy_score(y_valid,y_pre_v)
            pd.DataFrame(select).to_csv('so_'+data+'.csv')
            joblib.dump(svc_clf,'so_'+data+'.dat')
                
        fitness[i]=accuracy_score(y_test,y_pre)
    return np.array(fitness).reshape(-1,1)
                                
"""============================变量设置============================"""
length1= 1350 
length2= 1650
Lind = 80                                 # 染色体长度
data = ""
best = []
best_acc = 0  
"""=========================遗传算法参数设置========================="""
NIND = 20;                                 # 种群个体数目
MAXGEN = 200;                              # 最大遗传代数
def main():
    start_time = time.time()                             # 开始计时
    global data,best,best_acc
    best = []
    best_acc = 0
    FieldD = ea.crtbp(NIND,Lind)              # 调用函数创建区域描述器         
    Chrom = FieldD                           # 生成种群染色体矩阵
    obj_trace = np.zeros((MAXGEN, 2))         # 定义目标函数值记录器
    var_trace = np.zeros((MAXGEN, Lind))      # 定义染色体记录器，记录每一代最优个体的染色体
    """=========================开始遗传算法进化========================"""
    ObjV = aim(Chrom)                                     # 计算初始种群个体的目标函数值
    best_ind = np.argmax(ObjV)                             # 计算当代最优个体的序号
    for gen in range(MAXGEN):
        FitnV = ea.ranking(-ObjV)                          # 根据目标函数大小分配适应度值(由于遵循目标最小化约定，因此最大化问题要对目标函数值乘上-1)
        SelCh=Chrom[ea.selecting('rws', FitnV, NIND-1), :] # 选择，采用'rws'轮盘赌选择
        SelCh=ea.recombin('xovsp', SelCh, 0.7)             # 重组(采用两点交叉方式，交叉概率为0.7)
        SelCh=ea.mutbin('BG',SelCh)                        # 二进制种群变异
        # 把父代精英个体与子代合并
        Chrom = np.vstack([Chrom[best_ind, :], SelCh])
        ObjV = aim(Chrom)
        # 记录
        best_ind = np.argmax(ObjV)                       # 计算当代最优个体的序号
        obj_trace[gen, 0] = np.sum(ObjV) / NIND          # 记录当代种群的目标函数均值
        obj_trace[gen, 1] = ObjV[best_ind]               # 记录当代种群最优个体目标函数值
        best.append(best_acc)
        var_trace[gen, :] = Chrom[best_ind, :]           # 记录当代种群最优个体的变量值
    # 进化完成
    end_time = time.time() # 结束计时
    """============================输出结果及绘图================================"""
    best_gen = np.argmax(obj_trace[:, [1]])
    print('目标函数最大值：', obj_trace[best_gen, 1]) # 输出目标函数最大值
    pd.DataFrame(best).to_csv("so_fitness"+data+".csv")
    f = open ('./实验结果/SO_Time.txt','a')
    print(data+" 用时：\t",end_time-start_time,file=f)
    f.close()
   

d:\anconda\lib\site-packages\xgboost\compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
stock_list = ["HangSeng","CSI300","DJIA"]
for item in stock_list:
    print(item)
    data = item
    main()


HangSeng
TEST_ACC: 63.211
VALID_ACC: 74.762
******************************
TEST_ACC: 67.559
VALID_ACC: 81.905
******************************
TEST_ACC: 64.214
VALID_ACC: 73.333
******************************
TEST_ACC: 66.890
VALID_ACC: 80.476
******************************
TEST_ACC: 65.886
VALID_ACC: 70.476
******************************
TEST_ACC: 67.559
VALID_ACC: 66.667
******************************
TEST_ACC: 65.217
VALID_ACC: 77.143
******************************
TEST_ACC: 70.234
VALID_ACC: 77.619
******************************
TEST_ACC: 71.906
VALID_ACC: 76.190
******************************
TEST_ACC: 70.569
VALID_ACC: 81.429
******************************
TEST_ACC: 68.562
VALID_ACC: 73.333
******************************
TEST_ACC: 67.224
VALID_ACC: 70.000
******************************
TEST_ACC: 65.552
VALID_ACC: 77.143
******************************
TEST_ACC: 68.562
VALID_ACC: 79.048
******************************
TEST_ACC: 72.910
VALID_ACC: 80.000
******************************
T